# **ĐỒ ÁN: NHẬP MÔN KHOA HỌC DỮ LIỆU** #

### Nhóm 7 ###
|<div style="width:150px">MSSV</div>|<div style="width:290px">Tên</div>|
|---------- |:-------------:|
| 20120041  |  Trần Kim Bảo |
| 20120053  |    Nguyễn Thành Đạt   | 
| 20120071  | Nguyễn Thị Bích Hà |
| 20120113  | Lê Nguyên Khang |

## <font color='orange'> Import thư viện </font> ##

In [1]:
!pip install scrapy
!pip install spider3

In [1]:
import json
import re
import pandas as pd
import numpy as np
import scrapy

## <font color='orange'> Mô tả trang web </font> ##

Trong đồ án này nhóm mình xin được giới thiệu đến mọi người danh sách phim hay nhất mọi thời đại *(Thời gian tính đến ngày 19/11/2022)* được trang **IMDb** tổng hợp.

**IMDb** (<font color='blue'>https://www.imdb.com/</font>)  là viết tắt của **Internet Movie Database**, dịch theo tiếng Anh có nghĩa là Cơ sở dữ liệu điện ảnh trên Internet. Đây là một chuyên trang trực tuyến thông tin về phim điện ảnh trên toàn thế giới. Nó cung cấp toàn bộ các dữ liệu chi tiết về phim, diễn viên, đạo diễn, hãng sản xuất, bao gồm cả những người, công ty trong lĩnh vực sản xuất phim ảnh, phim truyền hình, video game.

**IMDb** còn là nơi cung cấp các đánh giá, phê bình uy tín dành cho các tín đồ nghiện phim ảnh. Các chuyên gia, người xem có thể đưa ra những góc nhìn cá nhân, các bình luận khen chê, đánh giá về mặt kỹ thuật, bối cảnh, kịch bản, hiệu ứng hình ảnh, công nghệ trình chiếu,… của các bộ phim dựa trên thang điểm 10. Dựa vào đó, người xem có thể tham khảo và quyết định có nên xem phim đó hay không.

Những thông tin được thu thập tại website: <font color='blue'>https://www.imdb.com/list/ls006266261/</font>




### Tạo một project mới với scrapy ###

In [2]:
!scrapy startproject film_crawler

Error: scrapy.cfg already exists in /home/bha/NM KHDL/NMKHDL_Project-20120041_KimBao/film_crawler


In [3]:
cd film_crawler/film_crawler

/home/bha/NM KHDL/NMKHDL_Project-20120041_KimBao/film_crawler/film_crawler


## <font color='orange'> Thu thập thông tin từ web </font> ##
**<h5>1. Thu thập url, id, votes, gross của các phim </h5>**

In [8]:
%%writefile spiders/collect_film_urls.py
import scrapy

class collect_film_url(scrapy.Spider):
  name='film_urls' 
  
  def start_requests(self):
    urls=['https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1']
    self.page=1
    yield scrapy.Request(url=urls[0], callback=self.parse)
        
  def parse(self, response):
    top=response.css('div.lister-list span.lister-item-index.unbold.text-primary::text').getall()
    id=response.css('div.lister-list img::attr(data-tconst)').getall()
    url=response.css('div.lister-list div.lister-item.mode-detail div.lister-item-image.ribbonize a::attr(href)').re(r'.*/title/.*\d{4,10}.')
    for i in range(len(top)):
      yield{
        'Top': top[i],
        'ID':id[i],
        'URL':url[i],
        'Votes':response.xpath(f'//*[@id="main"]/div/div[4]/div[3]/div[{i+1}]/div[2]/p[4]/span[2]/@data-value').get(),
        'Gross':response.xpath(f'//*[@id="main"]/div/div[4]/div[3]/div[{i+1}]/div[2]/p[4]/span[5]/@data-value').get()
      }
    if self.page<11:
      self.page+=1
      next_page_url=f'https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page={self.page}'
      yield scrapy.Request(url=next_page_url, callback=self.parse)
      

Writing spiders/collect_film_urls.py


- **Xuất các thông tin đã thu thập ở trên ra file json**

In [4]:
!scrapy crawl film_urls -o dataset/film_urls.json

2022-12-07 17:21:05 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: film_crawler)
2022-12-07 17:21:05 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.7.0, w3lib 2.0.1, Twisted 22.10.0, Python 3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) - [GCC 9.3.0], pyOpenSSL 22.1.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 38.0.2, Platform Linux-5.15.0-56-generic-x86_64-with-glibc2.10
2022-12-07 17:21:05 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'film_crawler',
 'NEWSPIDER_MODULE': 'film_crawler.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['film_crawler.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2022-12-07 17:21:05 [asyncio] DEBUG: Using selector: EpollSelector
2022-12-07 17:21:05 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2022-12-07 17:21:05 [scrapy.utils.log] 

2022-12-07 17:21:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '17.', 'ID': 'tt0116282', 'URL': '/title/tt0116282/', 'Votes': '678092', 'Gross': '24,611,975'}
2022-12-07 17:21:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '18.', 'ID': 'tt0468569', 'URL': '/title/tt0468569/', 'Votes': '2640997', 'Gross': '534,858,444'}
2022-12-07 17:21:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '19.', 'ID': 'tt0133093', 'URL': '/title/tt0133093/', 'Votes': '1906175', 'Gross': '171,479,930'}
2022-12-07 17:21:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '20.', 'ID': 'tt0175880', 'URL': '/title/tt0175880/', 'Votes': '

2022-12-07 17:21:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '49.', 'ID': 'tt0082971', 'URL': '/title/tt0082971/', 'Votes': '964045', 'Gross': '248,159,971'}
2022-12-07 17:21:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '50.', 'ID': 'tt0100150', 'URL': '/title/tt0100150/', 'Votes': '136221', 'Gross': '5,080,409'}
2022-12-07 17:21:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '51.', 'ID': 'tt0120601', 'URL': '/title/tt0120601/', 'Votes': '337045', 'Gross': '22,858,926'}
2022-12-07 17:21:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '52.', 'ID': 'tt0095016', 'URL': '/title/tt0095016/', 'Votes': '8753

2022-12-07 17:21:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '81.', 'ID': 'tt0401792', 'URL': '/title/tt0401792/', 'Votes': '770173', 'Gross': '74,103,820'}
2022-12-07 17:21:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '82.', 'ID': 'tt0387564', 'URL': '/title/tt0387564/', 'Votes': '423652', 'Gross': '56,000,369'}
2022-12-07 17:21:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '83.', 'ID': 'tt0097937', 'URL': '/title/tt0097937/', 'Votes': '75052', 'Gross': '14,743,391'}
2022-12-07 17:21:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1>
{'Top': '84.', 'ID': 'tt0310793', 'URL': '/title/tt0310793/', 'Votes': '14515

2022-12-07 17:21:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=2>
{'Top': '113.', 'ID': 'tt0114746', 'URL': '/title/tt0114746/', 'Votes': '618854', 'Gross': '57,141,459'}
2022-12-07 17:21:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=2>
{'Top': '114.', 'ID': 'tt0072890', 'URL': '/title/tt0072890/', 'Votes': '258193', 'Gross': '50,000,000'}
2022-12-07 17:21:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=2>
{'Top': '115.', 'ID': 'tt0094226', 'URL': '/title/tt0094226/', 'Votes': '311175', 'Gross': '76,270,454'}
2022-12-07 17:21:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=2>
{'Top': '116.', 'ID': 'tt0075860', 'URL': '/title/tt0075860/', 'Votes': '

...

2022-12-07 17:23:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=9>
{'Top': '875.', 'ID': 'tt2545118', 'URL': '/title/tt2545118/', 'Votes': '68336', 'Gross': '2,073,582'}
2022-12-07 17:23:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=9>
{'Top': '876.', 'ID': 'tt3659388', 'URL': '/title/tt3659388/', 'Votes': '852534', 'Gross': '228,433,663'}
2022-12-07 17:23:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=9>
{'Top': '877.', 'ID': 'tt1313104', 'URL': '/title/tt1313104/', 'Votes': '49869', 'Gross': '869,730'}
2022-12-07 17:23:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=9>
{'Top': '878.', 'ID': 'tt3850214', 'URL': '/title/tt3850214/', 'Votes': '86076

2022-12-07 17:23:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '906.', 'ID': 'tt0099077', 'URL': '/title/tt0099077/', 'Votes': '142987', 'Gross': '52,096,475'}
2022-12-07 17:23:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '907.', 'ID': 'tt1625346', 'URL': '/title/tt1625346/', 'Votes': '84819', 'Gross': '16,311,571'}
2022-12-07 17:23:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '908.', 'ID': 'tt0060176', 'URL': '/title/tt0060176/', 'Votes': '63143', 'Gross': None}
2022-12-07 17:23:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '909.', 'ID': 'tt0070334', 'URL': '/title/tt0070334/', 'Votes': '32068'

2022-12-07 17:23:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '938.', 'ID': 'tt0319061', 'URL': '/title/tt0319061/', 'Votes': '442413', 'Gross': '66,257,002'}
2022-12-07 17:23:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '939.', 'ID': 'tt0122933', 'URL': '/title/tt0122933/', 'Votes': '155504', 'Gross': '106,885,658'}
2022-12-07 17:23:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '940.', 'ID': 'tt0052618', 'URL': '/title/tt0052618/', 'Votes': '240047', 'Gross': '74,700,000'}
2022-12-07 17:23:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '941.', 'ID': 'tt0088286', 'URL': '/title/tt0088286/', 'Vote

2022-12-07 17:23:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '970.', 'ID': 'tt0070707', 'URL': '/title/tt0070707/', 'Votes': '43331', 'Gross': '2,905,334'}
2022-12-07 17:23:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '971.', 'ID': 'tt0047478', 'URL': '/title/tt0047478/', 'Votes': '346115', 'Gross': '269,061'}
2022-12-07 17:23:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '972.', 'ID': 'tt2870648', 'URL': '/title/tt2870648/', 'Votes': '52564', 'Gross': '8,413,144'}
2022-12-07 17:23:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=10>
{'Top': '973.', 'ID': 'tt3464902', 'URL': '/title/tt3464902/', 'Votes': '257

2022-12-07 17:23:31 [scrapy.core.engine] INFO: Closing spider (finished)
2022-12-07 17:23:31 [scrapy.extensions.feedexport] INFO: Stored json feed (1000 items) in: dataset/film_urls.json
2022-12-07 17:23:31 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 5978,
 'downloader/request_count': 12,
 'downloader/request_method_count/GET': 12,
 'downloader/response_bytes': 23642280,
 'downloader/response_count': 12,
 'downloader/response_status_count/200': 12,
 'elapsed_time_seconds': 146.430197,
 'feedexport/success_count/FileFeedStorage': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2022, 12, 7, 10, 23, 31, 956392),
 'item_scraped_count': 1000,
 'log_count/DEBUG': 1019,
 'log_count/INFO': 13,
 'memusage/max': 337485824,
 'memusage/startup': 103329792,
 'request_depth_max': 10,
 'response_received_count': 12,
 'robotstxt/request_count': 1,
 'robotstxt/response_count': 1,
 'robotstxt/response_status_count/200': 1,
 'scheduler/dequeued': 1

In [10]:
%%writefile spiders/collect_data_films.py
import scrapy
import pandas as pd
import numpy as np
from selenium import webdriver

class collect_data_film(scrapy.Spider):
  name='data_film'
  def __init__(self):
    try:
      df = pd.read_json('./dataset/film_urls.json', encoding='utf-8-sig')
      self.urls=df['URL']
    except IOError:
      print("File not found")
  def start_requests(self):
    self.headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:48.0) Gecko/20100101 Firefox/48.0'}
    for i in range(1000):
      next_page_url=f'https://www.imdb.com{self.urls[i]}?ref_=ttls_li_tt'
      yield scrapy.Request(url=next_page_url, callback=self.parse,headers=self.headers)

  def parse(self, response):

    # ID
    id_film=response.css('a.ipc-lockup-overlay.ipc-focusable::attr(href)').re(r'\w*\d{4,10}')[0]
    # Name
    name=response.css('h1[data-testid="hero-title-block__title"]::text').get()
    # Year release
    published_year=response.css("a[href*=releaseinfo]::text").re(r'\d{4}')[0]
    # Rate age
    age_rated=response.css("a[href*=certificates]::text").get()
    # Duration
    duration=response.css('div.ipc-metadata-list-item__content-container::text').getall()
    # Genres
    genres=response.css('span.ipc-chip__text::text').getall()

    # Director
    Director=set(response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]').xpath(".//*[contains(text(),'Director')]").xpath('following-sibling::div/ul/li/a/text()').getall())
    # Writes
    Writers=set(response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]').xpath(".//*[contains(text(),'Writer')]").xpath('following-sibling::div/ul/li/a/text()').getall())
    # Stars
    Stars=set(response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]').xpath(".//*[contains(text(),'Star')]").xpath('following-sibling::div/ul/li/a/text()').getall())
    
    # Budget
    try:
      Budget=response.xpath(".//*[contains(text(),'Budget')]").xpath('following-sibling::div/ul/li/label/text()').re(r'^.?.*\s')[0]
    except:
      Budget=None

    #  IMDb rating
    IMDb_rating = response.css("span.sc-7ab21ed2-1::text").get()
    # Popularity
    popularity = response.css("div.sc-edc76a2-1::text").get()
    # User reviews
    user_reviews = response.css("span.three-Elements:contains(User\ reviews) span::text").get()
    # Critic reviews
    critic_reviews = response.css("span.three-Elements:contains(Critic\ reviews) span::text").get()
    # Metascore
    meta_score = response.css("span.score-meta::text").get()
    # Oscar wins
    wins = response.css("a[href*=awards]::text").get()
    
    yield {
      'ID':id_film,
      'Name':name,
      'Published Year':published_year,
      'Rated':age_rated,
      'Duration':duration,
      'Genres':genres,
      
      'Director':Director,
      'Writers':Writers,
      'Stars':Stars,
      'Budget (estimated)':Budget,
      
      "IMDb RATING": IMDb_rating,
      "Popularity" : popularity,
      "User reviews": user_reviews,
      "Critic reviews": critic_reviews,
      "Meta score": meta_score,
      "Wins": wins,
    }

Overwriting spiders/collect_data_films.py


In [9]:
!scrapy crawl data_film -o dataset/data_film.json

2022-12-07 17:24:18 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: film_crawler)
2022-12-07 17:24:18 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.7.0, w3lib 2.0.1, Twisted 22.10.0, Python 3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) - [GCC 9.3.0], pyOpenSSL 22.1.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 38.0.2, Platform Linux-5.15.0-56-generic-x86_64-with-glibc2.10
2022-12-07 17:24:18 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'film_crawler',
 'NEWSPIDER_MODULE': 'film_crawler.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['film_crawler.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2022-12-07 17:24:18 [asyncio] DEBUG: Using selector: EpollSelector
2022-12-07 17:24:18 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2022-12-07 17:24:18 [scrapy.utils.log] 

2022-12-07 17:24:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0078788/?ref_=ttls_li_tt>
{'ID': 'tt0078788', 'Name': 'Apocalypse Now', 'Published Year': '1979', 'Rated': 'R', 'Duration': ['2', ' ', 'hours', ' ', '27', ' ', 'minutes'], 'Genres': ['Drama', 'Mystery', 'War', 'Back to top'], 'Director': {'Francis Ford Coppola'}, 'Writers': {'Michael Herr', 'Francis Ford Coppola', 'John Milius'}, 'Stars': {'Robert Duvall', 'Marlon Brando', 'Martin Sheen'}, 'Budget (estimated)': '$31,500,000 ', 'IMDb RATING': '8.5', 'Popularity': '307', 'User reviews': '1.3K', 'Critic reviews': '302', 'Meta score': '94', 'Wins': 'Won 2 Oscars'}
2022-12-07 17:24:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0137523/?ref_=ttls_li_tt>
{'ID': 'tt0137523', 'Name': 'Fight Club', 'Published Year': '1999', 'Rated': 'R', 'Duration': ['2', ' ', 'hours', ' ', '19', ' ', 'minutes'], 'Genres': ['Drama', 'Back to top'], 'Director': {'David Fincher'}, 'Write

2022-12-07 17:24:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0468569/?ref_=ttls_li_tt>
{'ID': 'tt0468569', 'Name': 'The Dark Knight', 'Published Year': '2008', 'Rated': 'PG-13', 'Duration': ['2', ' ', 'hours', ' ', '32', ' ', 'minutes'], 'Genres': ['Action', 'Crime', 'Drama', 'Back to top'], 'Director': {'Christopher Nolan'}, 'Writers': {'David S. Goyer', 'Christopher Nolan', 'Jonathan Nolan'}, 'Stars': {'Heath Ledger', 'Aaron Eckhart', 'Christian Bale'}, 'Budget (estimated)': '$185,000,000 ', 'IMDb RATING': '9.0', 'Popularity': '147', 'User reviews': '8.3K', 'Critic reviews': '442', 'Meta score': '84', 'Wins': 'Won 2 Oscars'}
2022-12-07 17:24:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0175880/?ref_=ttls_li_tt> (referer: None)
2022-12-07 17:24:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0175880/?ref_=ttls_li_tt>
{'ID': 'tt0175880', 'Name': 'Magnolia', 'Published Year': '1999', 'Rat

2022-12-07 17:24:25 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0118715/?ref_=ttls_li_tt>
{'ID': 'tt0118715', 'Name': 'The Big Lebowski', 'Published Year': '1998', 'Rated': 'R', 'Duration': ['1', ' ', 'hour', ' ', '57', ' ', 'minutes'], 'Genres': ['Comedy', 'Crime', 'Back to top'], 'Director': {'Joel Coen', 'Ethan Coen'}, 'Writers': {'Joel Coen', 'Ethan Coen'}, 'Stars': {'John Goodman', 'Jeff Bridges', 'Julianne Moore'}, 'Budget (estimated)': '$15,000,000 ', 'IMDb RATING': '8.1', 'Popularity': '532', 'User reviews': '1.4K', 'Critic reviews': '205', 'Meta score': '71', 'Wins': 'Awards'}
2022-12-07 17:24:25 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0129387/?ref_=ttls_li_tt>
{'ID': 'tt0129387', 'Name': "There's Something About Mary", 'Published Year': '1998', 'Rated': 'R', 'Duration': ['1', ' ', 'hour', ' ', '59', ' ', 'minutes'], 'Genres': ['Comedy', 'Romance', 'Back to top'], 'Director': {'Peter Farrelly', 'Bobby Farrelly

2022-12-07 17:24:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0114369/?ref_=ttls_li_tt> (referer: None)
2022-12-07 17:24:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0128445/?ref_=ttls_li_tt>
{'ID': 'tt0128445', 'Name': 'Rushmore', 'Published Year': '1998', 'Rated': 'R', 'Duration': ['1', ' ', 'hour', ' ', '33', ' ', 'minutes'], 'Genres': ['Comedy', 'Drama', 'Romance', 'Back to top'], 'Director': {'Wes Anderson'}, 'Writers': {'Owen Wilson', 'Wes Anderson'}, 'Stars': {'Bill Murray', 'Jason Schwartzman', 'Olivia Williams'}, 'Budget (estimated)': '$9,000,000 ', 'IMDb RATING': '7.6', 'Popularity': '2,844', 'User reviews': '698', 'Critic reviews': '185', 'Meta score': '86', 'Wins': 'Awards'}
2022-12-07 17:24:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0114369/?ref_=ttls_li_tt>
{'ID': 'tt0114369', 'Name': 'Se7en', 'Published Year': '1995', 'Rated': 'R', 'Duration': ['2', ' ', 'hours', ' ', 

...

2022-12-07 17:27:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt1322269/?ref_=ttls_li_tt>
{'ID': 'tt1322269', 'Name': 'August: Osage County', 'Published Year': '2013', 'Rated': 'R', 'Duration': ['2', ' ', 'hours', ' ', '1', ' ', 'minute'], 'Genres': ['Comedy', 'Drama', 'Back to top'], 'Director': {'John Wells'}, 'Writers': {'Tracy Letts'}, 'Stars': {'Meryl Streep', 'Dermot Mulroney', 'Julia Roberts'}, 'Budget (estimated)': '$25,000,000 ', 'IMDb RATING': '7.2', 'Popularity': '3,080', 'User reviews': '342', 'Critic reviews': '315', 'Meta score': '58', 'Wins': 'Nominated for 2 Oscars'}
2022-12-07 17:27:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0101452/?ref_=ttls_li_tt>
{'ID': 'tt0101452', 'Name': "Bill & Ted's Bogus Journey", 'Published Year': '1991', 'Rated': 'PG', 'Duration': ['1', ' ', 'hour', ' ', '33', ' ', 'minutes'], 'Genres': ['Adventure', 'Comedy', 'Fantasy', 'Back to top'], 'Director': {'Peter Hewitt'}, 'Writer

2022-12-07 17:27:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0070640/?ref_=ttls_li_tt>
{'ID': 'tt0070640', 'Name': 'Save the Tiger', 'Published Year': '1973', 'Rated': 'R', 'Duration': ['1', ' ', 'hour', ' ', '40', ' ', 'minutes'], 'Genres': ['Drama', 'Back to top'], 'Director': {'John G. Avildsen'}, 'Writers': {'Steve Shagan'}, 'Stars': {'Jack Lemmon', 'Jack Gilford', 'Laurie Heineman'}, 'Budget (estimated)': '$1,000,000 ', 'IMDb RATING': '6.9', 'Popularity': None, 'User reviews': None, 'Critic reviews': None, 'Meta score': None, 'Wins': 'Won 1 Oscar'}
2022-12-07 17:27:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0095647/?ref_=ttls_li_tt> (referer: None)
2022-12-07 17:27:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0095647/?ref_=ttls_li_tt>
{'ID': 'tt0095647', 'Name': 'Mississippi Burning', 'Published Year': '1988', 'Rated': 'R', 'Duration': ['2', ' ', 'hours', ' ', '8', ' ', 'minute

2022-12-07 17:27:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0059742/?ref_=ttls_li_tt> (referer: None)
2022-12-07 17:27:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0058331/?ref_=ttls_li_tt> (referer: None)
2022-12-07 17:27:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt2543164/?ref_=ttls_li_tt> (referer: None)
2022-12-07 17:27:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt1637725/?ref_=ttls_li_tt>
{'ID': 'tt1637725', 'Name': 'Ted', 'Published Year': '2012', 'Rated': 'R', 'Duration': ['1', ' ', 'hour', ' ', '46', ' ', 'minutes'], 'Genres': ['Comedy', 'Back to top'], 'Director': {'Seth MacFarlane'}, 'Writers': {'Wellesley Wild', 'Seth MacFarlane', 'Alec Sulkin'}, 'Stars': {'Mark Wahlberg', 'Mila Kunis', 'Seth MacFarlane'}, 'Budget (estimated)': '$50,000,000 ', 'IMDb RATING': '6.9', 'Popularity': '687', 'User reviews': '715', 'Critic reviews': '464', 'Meta

2022-12-07 17:27:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt3896198/?ref_=ttls_li_tt>
{'ID': 'tt3896198', 'Name': 'Guardians of the Galaxy Vol. 2', 'Published Year': '2017', 'Rated': 'PG-13', 'Duration': ['2', ' ', 'hours', ' ', '16', ' ', 'minutes'], 'Genres': ['Action', 'Adventure', 'Comedy', 'Back to top'], 'Director': {'James Gunn'}, 'Writers': {'Dan Abnett', 'Andy Lanning', 'James Gunn'}, 'Stars': {'Zoe Saldana', 'Dave Bautista', 'Chris Pratt'}, 'Budget (estimated)': '$200,000,000 ', 'IMDb RATING': '7.6', 'Popularity': '167', 'User reviews': '1.1K', 'Critic reviews': '599', 'Meta score': '67', 'Wins': 'Nominated for 1 Oscar'}
2022-12-07 17:27:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt6412452/?ref_=ttls_li_tt>
{'ID': 'tt6412452', 'Name': 'The Ballad of Buster Scruggs', 'Published Year': '2018', 'Rated': 'R', 'Duration': ['2', ' ', 'hours', ' ', '13', ' ', 'minutes'], 'Genres': ['Comedy', 'Drama', 'Musical', 'B

2022-12-07 17:27:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0106856/?ref_=ttls_li_tt> (referer: None)
2022-12-07 17:27:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0106856/?ref_=ttls_li_tt>
{'ID': 'tt0106856', 'Name': 'Falling Down', 'Published Year': '1993', 'Rated': 'R', 'Duration': ['1', ' ', 'hour', ' ', '53', ' ', 'minutes'], 'Genres': ['Action', 'Crime', 'Drama', 'Back to top'], 'Director': {'Joel Schumacher'}, 'Writers': {'Ebbe Roe Smith'}, 'Stars': {'Robert Duvall', 'Michael Douglas', 'Barbara Hershey'}, 'Budget (estimated)': '$25,000,000 ', 'IMDb RATING': '7.6', 'Popularity': '1,362', 'User reviews': '523', 'Critic reviews': '96', 'Meta score': '56', 'Wins': 'Awards'}
2022-12-07 17:27:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt5463162/?ref_=ttls_li_tt> (referer: None)
2022-12-07 17:27:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt5463162/?r

## <font color='orange'> Tiền xử lý dữ liệu </font> ##

<h4>1. Đọc dữ liệu</h4>

- <h5>Đọc file film_urls vào dataframe<h5>

In [7]:
df_url=pd.read_json('dataset/film_urls.json',encoding='utf-8-sig')
df_url.head()

,Top,ID,URL,Votes,Gross
0,1.,tt0068646,/title/tt0068646/,1849028,"134,966,411"
1,2.,tt0099685,/title/tt0099685/,1156623,"46,836,394"
2,3.,tt0110912,/title/tt0110912/,2042817,"107,928,762"
3,4.,tt0114814,/title/tt0114814/,1083159,"23,341,568"
4,5.,tt0078788,/title/tt0078788/,666751,"83,471,511"


- <h5>Đọc file data_film vào dataframe</h5>

In [10]:
df_film=pd.read_json('dataset/data_film.json',encoding='utf-8-sig')
df_film.head()

,ID,Name,Published Year,Rated,Duration,Genres,Director,Writers,Stars,Budget (estimated),IMDb RATING,Popularity,User reviews,Critic reviews,Meta score,Wins
0,tt0068646,The Godfather,1972,R,"[2, , hours, , 55, , minutes]","[Crime, Drama, Back to top]",[Francis Ford Coppola],"[Mario Puzo, Francis Ford Coppola]","[James Caan, Marlon Brando, Al Pacino]","$6,000,000",9.2,92,5.2K,193,100.0,Won 3 Oscars
1,tt0099685,Goodfellas,1990,R,"[2, , hours, , 25, , minutes]","[Biography, Crime, Drama, Back to top]",[Martin Scorsese],"[Martin Scorsese, Nicholas Pileggi]","[Ray Liotta, Robert De Niro, Joe Pesci]","$25,000,000",8.7,194,1.5K,163,90.0,Won 1 Oscar
2,tt0114814,The Usual Suspects,1995,R,"[1, , hour, , 46, , minutes]","[Crime, Drama, Mystery, Back to top]",[Bryan Singer],[Christopher McQuarrie],"[Kevin Spacey, Gabriel Byrne, Chazz Palminteri]","$6,000,000",8.5,425,1.4K,154,77.0,Won 2 Oscars
3,tt0108052,Schindler's List,1993,R,"[3, , hours, , 15, , minutes]","[Biography, Drama, History, Back to top]",[Steven Spielberg],"[Steven Zaillian, Thomas Keneally]","[Ben Kingsley, Ralph Fiennes, Liam Neeson]","$22,000,000",9.0,219,2.1K,175,94.0,Won 7 Oscars
4,tt0110912,Pulp Fiction,1994,R,"[2, , hours, , 34, , minutes]","[Crime, Drama, Back to top]",[Quentin Tarantino],"[Roger Avary, Quentin Tarantino]","[Uma Thurman, John Travolta, Samuel L. Jackson]","$8,000,000",8.9,115,3.5K,304,94.0,Won 1 Oscar


- <h5>Hợp dữ liệu lại</h5>

In [11]:
film_info_df=df_url.merge(df_film, on="ID", how="outer")
film_info_df.shape

(1000, 20)

In [12]:
film_info_df.head()

,Top,ID,URL,Votes,Gross,Name,Published Year,Rated,Duration,Genres,Director,Writers,Stars,Budget (estimated),IMDb RATING,Popularity,User reviews,Critic reviews,Meta score,Wins
0,1.,tt0068646,/title/tt0068646/,1849028,"134,966,411",The Godfather,1972,R,"[2, , hours, , 55, , minutes]","[Crime, Drama, Back to top]",[Francis Ford Coppola],"[Mario Puzo, Francis Ford Coppola]","[James Caan, Marlon Brando, Al Pacino]","$6,000,000",9.2,92,5.2K,193,100.0,Won 3 Oscars
1,2.,tt0099685,/title/tt0099685/,1156623,"46,836,394",Goodfellas,1990,R,"[2, , hours, , 25, , minutes]","[Biography, Crime, Drama, Back to top]",[Martin Scorsese],"[Martin Scorsese, Nicholas Pileggi]","[Ray Liotta, Robert De Niro, Joe Pesci]","$25,000,000",8.7,194,1.5K,163,90.0,Won 1 Oscar
2,3.,tt0110912,/title/tt0110912/,2042817,"107,928,762",Pulp Fiction,1994,R,"[2, , hours, , 34, , minutes]","[Crime, Drama, Back to top]",[Quentin Tarantino],"[Roger Avary, Quentin Tarantino]","[Uma Thurman, John Travolta, Samuel L. Jackson]","$8,000,000",8.9,115,3.5K,304,94.0,Won 1 Oscar
3,4.,tt0114814,/title/tt0114814/,1083159,"23,341,568",The Usual Suspects,1995,R,"[1, , hour, , 46, , minutes]","[Crime, Drama, Mystery, Back to top]",[Bryan Singer],[Christopher McQuarrie],"[Kevin Spacey, Gabriel Byrne, Chazz Palminteri]","$6,000,000",8.5,425,1.4K,154,77.0,Won 2 Oscars
4,5.,tt0078788,/title/tt0078788/,666751,"83,471,511",Apocalypse Now,1979,R,"[2, , hours, , 27, , minutes]","[Drama, Mystery, War, Back to top]",[Francis Ford Coppola],"[Michael Herr, Francis Ford Coppola, John Milius]","[Robert Duvall, Marlon Brando, Martin Sheen]","$31,500,000",8.5,307,1.3K,302,94.0,Won 2 Oscars


##  2. Khám phá dữ liệu </font> ##

### Sắp xếp các cột dữ liệu

Hiện tại, các cột dữ liệu đang không theo một trật tự nhất định nào và khó quan sát. Để dễ dàng quan sát hơn, ta tiến hành sắp xếp lại thứ tự các cột.

In [13]:
film_info_df=film_info_df[['Top','ID','Name','Published Year','Rated','Duration','Genres','Director','Writers','Stars','IMDb RATING','Budget (estimated)','Gross','Popularity','Votes','User reviews','Critic reviews','Meta score','Wins','URL']]

In [14]:
film_info_df.head()

,Top,ID,Name,Published Year,Rated,Duration,Genres,Director,Writers,Stars,IMDb RATING,Budget (estimated),Gross,Popularity,Votes,User reviews,Critic reviews,Meta score,Wins,URL
0,1.,tt0068646,The Godfather,1972,R,"[2, , hours, , 55, , minutes]","[Crime, Drama, Back to top]",[Francis Ford Coppola],"[Mario Puzo, Francis Ford Coppola]","[James Caan, Marlon Brando, Al Pacino]",9.2,"$6,000,000","134,966,411",92,1849028,5.2K,193,100.0,Won 3 Oscars,/title/tt0068646/
1,2.,tt0099685,Goodfellas,1990,R,"[2, , hours, , 25, , minutes]","[Biography, Crime, Drama, Back to top]",[Martin Scorsese],"[Martin Scorsese, Nicholas Pileggi]","[Ray Liotta, Robert De Niro, Joe Pesci]",8.7,"$25,000,000","46,836,394",194,1156623,1.5K,163,90.0,Won 1 Oscar,/title/tt0099685/
2,3.,tt0110912,Pulp Fiction,1994,R,"[2, , hours, , 34, , minutes]","[Crime, Drama, Back to top]",[Quentin Tarantino],"[Roger Avary, Quentin Tarantino]","[Uma Thurman, John Travolta, Samuel L. Jackson]",8.9,"$8,000,000","107,928,762",115,2042817,3.5K,304,94.0,Won 1 Oscar,/title/tt0110912/
3,4.,tt0114814,The Usual Suspects,1995,R,"[1, , hour, , 46, , minutes]","[Crime, Drama, Mystery, Back to top]",[Bryan Singer],[Christopher McQuarrie],"[Kevin Spacey, Gabriel Byrne, Chazz Palminteri]",8.5,"$6,000,000","23,341,568",425,1083159,1.4K,154,77.0,Won 2 Oscars,/title/tt0114814/
4,5.,tt0078788,Apocalypse Now,1979,R,"[2, , hours, , 27, , minutes]","[Drama, Mystery, War, Back to top]",[Francis Ford Coppola],"[Michael Herr, Francis Ford Coppola, John Milius]","[Robert Duvall, Marlon Brando, Martin Sheen]",8.5,"$31,500,000","83,471,511",307,666751,1.3K,302,94.0,Won 2 Oscars,/title/tt0078788/


### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột?

In [15]:
num_rows, num_cols = film_info_df.shape
num_rows, num_cols

(1000, 20)

**Nhận xét:** 
* Tập dữ liệu gồm 1000 dòng và 20 cột.

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

* Theo quan sát, mỗi dòng thể hiện thông tin chi tiết của các phim trong [top 1000 phim hay nhất mọi thời đại](https://www.imdb.com/list/ls006266261/), được [IMDb](https://www.imdb.com/?ref_=nv_home)  tổng hợp.
* Có vẻ **không** có dòng nào có ý nghĩa khác nhau. 

### Dữ liệu có các dòng bị lặp không?

Kiểm tra việc này và lưu kết quả vào biến `have_duplicated_rows`. Biến này sẽ có giá trị `True` nếu dữ liệu có các dòng bị lặp và có giá trị `False` nếu ngược lại. Do ở đây cột `ID` mang tính định danh cho từng phim nên ta sẽ kiểm tra xem thử có ID phim nào trùng nhau không.

In [17]:
check_duplicated = set(film_info_df.ID.duplicated())

if True in check_duplicated:
    have_duplicated_rows = True
else:
    have_duplicated_rows = False
    
if (have_duplicated_rows == True):
    print('Có dòng dữ liệu bị lặp')
else:
    print('Không có dòng dữ liệu nào bị lặp')

Không có dòng dữ liệu nào bị lặp


### Mỗi cột có ý nghĩa gì?

|Tên cột dữ liệu | Mô tả | Đơn vị |
|:------:| ---| -- |
|Top| Thể hiện thứ hạng của phim trên bảng xếp hạng |
|ID|  Mã định danh của phim|
|Name| Tên của phim|
|Published Year| Năm xuất bản phim| năm |
|Rated| Loại phim dựa trên Hệ thống phân loại phim của MPAA, để phân loại phim cho các đối tượng xem trẻ em, thanh thiếu niên và người lớn |  | 
|Duration| Thời lượng phim |  giờ
|Genres|Thể loại phim|  |
|Director|Tên các đạo diễn sản xuất phim| |
|Writers|Tên các biên kịch phim|  |
|Stars|Tên các diễn viên tham gia (ở cột này chỉ lấy tên các diễn viên được hiển thị trên trang web) |  |
|IMDb RATING|Điểm IMDb được tính theo thang điểm 10, nghĩa là nếu điểm số càng cao thì chất lượng càng tốt. | |
|Budget (estimated)|Kinh phí thực hiện phim (ước tính)| \$ |
|Gross|Doanh thu của phim tính đến thời điểm hiện tại| \$ |
|Popularity|Độ phổ biến của phim (thứ tự trên bảng xếp hạng) được tính đến tuần hiện tại|  |
|Votes|Số lượt bình chọn của người xem trên trang IMDb dành cho phim|  |
|User reviews| Số lượng nhận xét của người xem |  |
|Critic reviews|Số lượng nhận xét của các nhà phê bình |  |
|Meta score|Điểm trung bình của các nhà phê bình lấy từ bộ điểm số của Metacritic, với thang điểm là 100|  |
|Wins|Số lượng giải thưởng và đề cử mà phim đạt được trong Giải thưởng Viện Hàn lâm, thường được biết đến với tên Giải Oscar |
|URL| Địa chỉ truy cập vào trang web mô tả chi tiết của phim|



* **Thời điểm hiện tại** là tính đến ngày 12/11/2022

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [22]:
col_dtypes = film_info_df.dtypes
col_dtypes

Top                    object
ID                     object
Name                   object
Published Year          int64
Rated                  object
Duration               object
Genres                 object
Director               object
Writers                object
Stars                  object
IMDb RATING           float64
Budget (estimated)     object
Gross                  object
Popularity             object
Votes                   int64
User reviews           object
Critic reviews         object
Meta score            float64
Wins                   object
URL                    object
dtype: object

Ta thấy ở cột `Top`, `Gross` đang ở dạng `object`, nhưng đúng là ở dạng `numeric`, nên ta thực hiện tiền xử lý về đúng kiểu dữ liệu.

In [23]:
film_info_df.Top = film_info_df.Top.replace(',','',regex = True).replace('\.','',regex = True).astype(int)
film_info_df.Gross = film_info_df.Gross.replace([",",None],['',np.nan],regex=True).astype(float)

<h1>MỌI NGƯỜI XEM THỬ CÒN PHẦN NÀO CHƯA XỬ LÝ THÌ XỬ LÝ NHA</h1>

In [53]:
film_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Top                 1000 non-null   int64  
 1   ID                  1000 non-null   object 
 2   Name                1000 non-null   object 
 3   Published Year      1000 non-null   int64  
 4   Rated               997 non-null    object 
 5   Duration            1000 non-null   object 
 6   Genres              1000 non-null   object 
 7   Director            1000 non-null   object 
 8   Writers             1000 non-null   object 
 9   Stars               1000 non-null   object 
 10  IMDb RATING         1000 non-null   float64
 11  Budget (estimated)  895 non-null    object 
 12  Gross               937 non-null    float64
 13  Popularity          782 non-null    object 
 14  Votes               1000 non-null   int64  
 15  User reviews        963 non-null    object 
 16  Critic 